In [2]:
import torch 
import torchaudio
from torch import nn
from torch.utils.data import DataLoader

from RAVDESSdataset import EmotionSpeechDataset
from cnn import CNNNetwork
# https://github.com/musikalkemist/pytorchforaudio/blob/main/09%20Training%20urban%20sound%20classifier/train.py

BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001

ANNOTATIONS_FILE = '/Users/stephen/Emotion_Dectection/data/RAVDESS/metadata_class.csv'
AUDIO_DIR = '/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24'
SAMPLE_RATE = 16000
NUM_SAMPLES = 16000


def create_data_loader(train_data, batch_size):
    train_dataloader = DataLoader(train_data, batch_size=batch_size)
    return train_dataloader


def train_single_epoch(model, data_loader, loss_fn, optimiser, device):
    for input, target in data_loader:
        input, target = input.to(device), target.to(device)

        # calculate loss
        prediction = model(input)
        loss = loss_fn(prediction, target)

        # backpropagate error and update weights
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

    print(f"loss: {loss.item()}")


def train(model, data_loader, loss_fn, optimiser, device, epochs):
    for i in range(epochs):
        print(f"Epoch {i+1}")
        train_single_epoch(model, data_loader, loss_fn, optimiser, device)
        print("---------------------------")
    print("Finished training")


if __name__ == "__main__":
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    print(f"Using {device}")

    # instantiating our dataset object and create data loader
    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=SAMPLE_RATE,
        n_fft=1024,
        hop_length=512,
        n_mels=64
    )

    emo = EmotionSpeechDataset(ANNOTATIONS_FILE,
                            AUDIO_DIR,
                            mel_spectrogram,
                            SAMPLE_RATE,
                            NUM_SAMPLES,
                            device)
    
    train_dataloader = create_data_loader(emo, BATCH_SIZE)

    # construct model and assign it to device
    cnn = CNNNetwork().to(device)
    print(cnn)

    # initialise loss funtion + optimiser
    loss_fn = nn.CrossEntropyLoss()
    optimiser = torch.optim.Adam(cnn.parameters(),
                                 lr=LEARNING_RATE)

    # train model
    train(cnn, train_dataloader, loss_fn, optimiser, device, EPOCHS)

    # save model
    torch.save(cnn.state_dict(), "feedforwardnet.pth")
    print("Trained feed forward net saved at feedforwardnet.pth")



Using cpu
CNNNetwork(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv4): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=2560, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)
Epoch 1


TypeError: join() argument must be str, bytes, or os.PathLike object, not 'int64'

In [5]:
# Turning dataset into Pytorch Dataset object 
import pandas as pd
import torch 
import torchaudio
from torch import nn
from torch.utils.data import DataLoader

from RAVDESSdataset import EmotionSpeechDataset
from cnn import CNNNetwork
# https://github.com/musikalkemist/pytorchforaudio/blob/main/09%20Training%20urban%20sound%20classifier/train.py

BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001

ANNOTATIONS_FILE = '/Users/stephen/Emotion_Dectection/data/RAVDESS/metadata_class.csv'
AUDIO_DIR = '/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24'
SAMPLE_RATE = 16000
NUM_SAMPLES = 16000








from torch.utils.data import Dataset

class EmotionSpeechDataset(Dataset):

    def __init__(self, annotations_file, audio_dir, transformation, target_sample_rate, num_samples, device):
        self.annotations = pd.read_csv(annotations_file, index_col=1)
        self.audio_dir = audio_dir
        self.device = device 
        self.transformation = transformation.to(self.device)
        self.target_sample_rate = target_sample_rate
        self.num_samples = num_samples 

    def __len__(self):
        return len(self.annotations)


    def __getitem__(self, index):
        audio_sample_path = self._get_audio_sample_path(index)
        label = self._get_audio_sample_label(index)
        signal, sr = torchaudio.load(audio_sample_path)
        signal = signal.to(self.device)
        signal = self._resample_if_necessary(signal, sr)
        signal = self._mix_down_if_necessary(signal)
        signal = self._cut_if_necessary(signal)
        signal = self._right_pad_if_necessary(signal)
        signal = self.transformation(signal) # Mel-Spectrum 
        return signal, label


    def _cut_if_necessary(self, signal):
        # signal -> tensor -> (1, num_samples) -> (1, 50000) -> (1, 16000)
        if signal.shape[1] > self.num_samples: 
            signal = signal[:, :self.num_samples]
        return signal 

    def _right_pad_if_necessary(self, signal):
        length_signal = signal.shape[1]
        if length_signal < self.num_samples:
            num_missing_samples = self.num_samples - length_signal
            last_dim_padding = (0, num_missing_samples)
            signal = torch.nn.functional.pad(signal, last_dim_padding)
        return signal

    def _resample_if_necessary(self, signal, sr):
        if sr != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(sr, self.target_sample_rate)
            signal = resampler(signal)
        return signal

    def _mix_down_if_necessary(self, signal):
        if signal.shape[0] > 1:
            signal = torch.mean(signal, dim=0, keepdim=True)
        return signal

    def _get_audio_sample_path(self, index):
        fold = str(self.annotations.iloc[index, 4])
        wav = str(self.annotations.iloc[index, 1])
        path = os.path.join(self.audio_dir, fold, wav)
        return path                  
        
 

    def _get_audio_sample_label(self, index):
        return self.annotations.iloc[index, 2]

if __name__ == "__main__":
    ANNOTATIONS_FILE = "/Users/stephen/Emotion_Dectection/data/RAVDESS/metadata_class.csv"
    AUDIO_DIR = "/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24"
    SAMPLE_RATE = 22050
    NUM_SAMPLES = 22050

    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    print(f"Using device {device}")

    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=SAMPLE_RATE,
        n_fft=1024,
        hop_length=512,
        n_mels=64
    )

    emo = EmotionSpeechDataset(ANNOTATIONS_FILE, AUDIO_DIR, mel_spectrogram, SAMPLE_RATE, NUM_SAMPLES, device )
    print(f"There are {len(emo)} samples in the dataset.")
    signal, label = emo[0]

Using device cpu
There are 1440 samples in the dataset.


formats: can't open input file `/03-01-08-02-02-01-01.wav': No such file or directory


RuntimeError: Error loading audio file: failed to open file /03-01-08-02-02-01-01.wav

In [ ]:
fold = f"fold{self.annotations.iloc[index, 5]}"